In [230]:
import numpy as np
import project
import scale

From page 426 in the Bertsikas textbook there is an algorithm that we are testing here 

In [231]:
def generate_A(n):
    A = -100*np.random.rand(n,n)
    return A

In [232]:
# initial values 

x_0 = np.matrix([[0.5],[0.5]])
s_0 = np.matrix([[1],[1]])
p_0 = np.matrix([[0],[1]])

In [239]:
A = np.matrix([[1,1],
               [1,0]])
x = x_0
s = s_0
p = p_0
epsilon = 10**-3
k = 0
alpha = 0.9
mu = 100000
c = np.matrix([[1],[2]])

# cannot divide by 0.0, so need to check for that.  This is slightly inaccurate because if x 
#...is close to zero and negative then it will not preserve the negative (i.e. it puts it to 10**6)

udf = lambda x: (x)**-2 if x!=0.0 else 10**6
vfunc = np.vectorize(udf)

udf_1 = lambda x: (x)**-1 if x!=0.0 else 10**3
vfunc_1 = np.vectorize(udf_1)

# collect the x_s
x_s = []
d_s = []

while s.T*x > epsilon and k < 100:
    
    X = np.diag(x.T.tolist()[0])
    mu = alpha*mu

    aug_A_1, aug_b_1 = get_A_b(mu, X, A, e, c)
    aug_A_2, aug_b_2 = get_A_b_2(A)

    # stack the A's
    A_solv = np.vstack((aug_A_1, aug_A_2))

    # stack the b's
    b_solv = np.vstack((aug_b_1, aug_b_2))
    
    d_p = np.linalg.solve(A_solv, b_solv)
    # extract d and p
    # d is the first "n" elements
    d = d_p[:A.shape[1]]
    # p is the last "m" elements
    p = d_p[-A.shape[0]:]

    # update the solutions and change x by "d"
    x_s = x_s + [x]
    d_s = d_s + [d]
    
    x = x + d
    p = p
    # this is making s negative 
    s = c - A.T*p
    
    k += 1
    
print 'done, and s.T*x is ', s.T*x, 'and k is ', k

done, and s.T*x is  [[-90179997.]] and k is  1


In [241]:
d_s

[matrix([[  1.65569120e-19],
         [ -1.65568458e-19]])]

In [242]:
x_s

[matrix([[ 0.5],
         [ 0.5]])]

### need to unpack the matrices and form Ax=b problem

In [181]:
# need to solve for p and d 
# p are the costs for the objective function of the dual problem 
# d are the directions that we will move in. They are expected to be small, I think.  
# They are the directions that minimize the Taylor series expansion of the Barrier function

# need the matrix A and b for Ax=b

# need to do a horizontal concatenation here to make one augmented matrix.
# aug_A_1 = np.hstack((np.asmatrix(mu*vfunc(X)), A.T))
# aug_b_1 = np.asmatrix(mu*vfunc_1(X))*np.asmatrix(e).T - c

# modularize this

def get_A_b(mu, X, A, e, c):
    aug_A_1 = np.hstack((np.asmatrix(mu*vfunc(X)), A.T))
    aug_b_1 = np.asmatrix(mu*vfunc_1(X))*np.asmatrix(e).T - c
    return aug_A_1, aug_b_1

# get second part 

def get_A_b_2(A):
    aug_A_2 = np.hstack((np.asmatrix(A), np.zeros((A.shape[0],A.shape[0]))))
    aug_b_2 = np.asmatrix(np.zeros((A.shape[0],1)))
    return aug_A_2, aug_b_2

In [182]:
aug_A_1, aug_b_1 = get_A_b(mu, X, A, e, c)
aug_A_2, aug_b_2 = get_A_b_2(A)

In [188]:
# stack the A's
A_solv = np.vstack((aug_A_1, aug_A_2))

# stack the b's
b_solv = np.vstack((aug_b_1, aug_b_2))

In [191]:
x_solv = np.linalg.solve(A_solv, b_solv)

In [235]:
x_solv

matrix([[ -1.36964880e-24],
        [  3.42412200e-19],
        [  2.65945217e+03],
        [  1.00000000e+00]])

Figure out why s'x is negative?

### Feasible initial point

This is covered on page 430 in the textbook.  To find a feasible initial point, we need to solve a different linear program.